In [ ]:
%%capture
# Transformers installation
! pip install datasets
! pip install transformers[torch]
! pip install accelerate
! pip install evaluate
! pip install sacrebleu
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

#  Thực hiện load bộ dataset

Để demo, code dưới đây chỉ load 2% dữ liệu huấn luyện và training. Nếu muốn load full thì bỏ đi option split.


In [ ]:
%%capture
from datasets import load_dataset
import datasets

mt_dataset = load_dataset("mt_eng_vietnamese", "iwslt2015-vi-en", split={
    'test': datasets.ReadInstruction('test', to=2, unit='%'),
    'train': datasets.ReadInstruction('train', to=2, unit='%')
})


In [ ]:
mt_dataset

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 25
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 2666
    })
})

# Tiền xử lý

**Tiền xử lý bao gồm các bước sau**:

1. Thêm tiền tố vào đầu đầu vào với một câu gợi ý (prompt) để mô hình biết đây là một nhiệm vụ dịch. Một số mô hình có khả năng thực hiện nhiều nhiệm vụ NLP sẽ cần câu gợi ý (prompting) cho từng nhiệm vụ cụ thể.

2. Tokenize chuỗi đầu vào (tiếng Việt) và đầu ra (tiếng Anh) riêng biệt vì không thể tách từng phần của văn bản tiếng Việt bằng một tokenizer được huấn luyện trước trên từ vựng tiếng Anh.

3. Cắt giữa các chuỗi sao cho chúng không dài hơn độ dài tối đa được đặt bởi tham số max_length.

In [ ]:
# @title Load tokenizer
variable_name = "VietAI/vit5-base" # @param ["VietAI/vit5-base", "VietAI/vit5-large", "minhtoan/t5-small-wikilingua-vietnamese"]
from transformers import AutoTokenizer

# checkpoint = "t5-small"
checkpoint="VietAI/vit5-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
source_lang = "vi"
target_lang = "en"
prefix = "dịch đoạn sau sang Tiếng Anh: "


def preprocess_function(examples):
    inputs = [prefix + example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

Để áp dụng chức năng tiền xử lý trên toàn bộ bộ dữ liệu, sử dụng phương thức [map](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.map). Bạn có thể tăng tốc độ của hàm `map` bằng cách đặt `batched=True` để xử lý theo batch.

In [ ]:
tokenized_mt_dataset = mt_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/2666 [00:00<?, ? examples/s]

Tạo một batch bằng cách sử dụng [DataCollatorForSeq2Seq](https://huggingface.co/docs/transformers/main/en/main_classes/data_collator#transformers.DataCollatorForSeq2Seq). Việc padding các câu thành với max_lengt là độ dài câu dài nhất trong batch đó hiệu quả hơn là padding toàn bộ bộ dữ liệu đến độ dài tối đa.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

# Hàm đánh giá

Sử dụng một hàm đánh giá trong quá trình huấn luyện thường rất hữu ích để đánh giá hiệu suất của mô hình một cách thường xuyên, giúp phát hiện được lỗi sớm, tránh lãng phí thời gian. Ta có thể nhanh chóng tải một phương pháp đánh giá bằng thư viện 🤗 [Evaluate](https://huggingface.co/docs/evaluate/index). Đối với bài toán dịch máy, sử dụng metric SacreBLEU (xem [hướng dẫn nhanh](https://huggingface.co/docs/evaluate/a_quick_tour) của 🤗 Evaluate để tìm hiểu thêm về cách tải và cách tính toán metric)

In [ ]:
%%capture
import evaluate
metric = evaluate.load("sacrebleu")

Sau đó, hãy tạo một hàm để truyền dự đoán và nhãn để hàm [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) tính toán điểm SacreBLEU:

In [ ]:
import numpy as np


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Huấn luyện mô hình

Các thành phần đã sẵn sàng, việc tiếp theo là load mô hình.

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

3 bước tiếp theo cần thực hiện để huấn luyện được mô hình:

1. Xác định các siêu tham số có trong class [Seq2SeqTrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments). Tham số bắt buộc duy nhất là `output_dir`, chỉ định nơi lưu trữ mô hình. Cuối mỗi epoch, Trainer sẽ đánh giá mô hình dựa trên độ đo `SacreBLEU` và lưu điểm kiểm tra của quá trình huấn luyện.

2. Truyền các tham số vừa config vào instance `Seq2SeqTrainer` kèm với model, dataset, tokenizer, data_collator và hàm compute_metrics.

3. Gọi train() để huấn luyện mô hình.

In [ ]:
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="my_translator_vi-en",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    fp16=True,
    # push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_mt_dataset["train"],
    eval_dataset=tokenized_mt_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,3.955700,3.535253,1.751800,18.160000
2,3.249000,3.372021,2.647200,18.240000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=1334, training_loss=3.4784665772582457, metrics={'train_runtime': 400.1466, 'train_samples_per_second': 13.325, 'train_steps_per_second': 3.334, 'total_flos': 331258820014080.0, 'train_loss': 3.4784665772582457, 'epoch': 2.0})

In [ ]:
trainer.save_model('./my_translator_vi-en-model')

# Dự đoán

Great, now that you've finetuned a model, you can use it for inference!

Come up with some text you'd like to translate to another language. For T5, you need to prefix your input depending on the task you're working on. For translation from English to French, you should prefix your input as shown below:

In [ ]:
text = "dịch đoạn sau sang Tiếng Anh: Tôi là sinh viên trường Đại học Công Nghệ, ĐHQGHN !"

## 1. Sử dụng API Pipline của hugging-face

The simplest way to try out your finetuned model for inference is to use it in a [pipeline()](https://huggingface.co/docs/transformers/main/en/main_classes/pipelines#transformers.pipeline). Instantiate a `pipeline` for translation with your model, and pass your text to it:

Cách đơn giản nhất để thử nghiệm mô hình sau khi huấn luyện là sử dụng nó trong một `pipeline()`. Tạo một `pipeline` cho task `trasnlation` với tham số thứ hai là đường dẫn tới mô hình vừa được lưu. Sau đó, truyền trực tiếp văn bản để cho mô hình thực hiện nhiệm vụ dịch.

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="/content/my_translator_vi-en-model")
translator(text)

Your input_length: 21 is bigger than 0.9 * max_length: 20. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


[{'translation_text': 'I &apos;m going to school of Technology , ĐHQGHN .'}]

## 2. Sử dụng API của mô hình

Bước đầu tiên, ta cần tokenize đoạn văn bản đầu vào và trả về `input_ids`.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/my_translator_vi-en-model")
inputs = tokenizer(text, return_tensors="pt").input_ids

Sử dụng hàm [generate()](https://huggingface.co/docs/transformers/main/en/main_classes/text_generation#transformers.GenerationMixin.generate) để thực hiện việc infer và trả về đầu ra. Để biết thêm chi tiết về các chiến lược tạo văn bản khác nhau và các tham số để điều khiển quá trình tạo ra văn bản, bạn có thể kiểm tra tại [Text Generation API](https://huggingface.co/docs/transformers/main/en/tasks/../main_classes/text_generation).

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/content/my_translator_vi-en-model")
outputs = model.generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

Decode `input_ids` đầu ra để thấy được chuỗi đầu ra.

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'I &apos;m not my student of Science and Technology!'

# Bài tập 1:  
Sử dụng code trên, áp dụng với bộ dữ liệu [Opus Books](https://huggingface.co/datasets/opus_books) để huấn luyện mô hình dịch máy Tiếng Anh sang Tiếng Pháp và đánh giá hiệu quả trên tập test.

# Bài tập 2:
Đề xuất và implment phương pháp để tạo ra mô hình dịch máy 2 chiều (từ VN sang EN và ngược lại). Đánh giá hiệu quả dịch của mô hình với 2 chiều dịch đó.